### Bayesian Naive bayes

The data set $D$ consists of $n$ pairs of $x$ and $y$

$$ D = ((x^{(1)}, y_1), ..., (x^{(n)}, y_n)) $$

$x^{(i)}$ is the counts of words in invoice $i$. There are $d$ different words in the data set.

$$ x^{(i)} = (x^{(i)}_1, ..., x^{(i)}_d) $$

$y_i$ is the sender of invoice $i$. There are $m$ different senders in the data set.

$$ y_i \in \{1,...,m\} $$

The distribution for the sender $y$ is a categorical distribution, parameterized by $\theta$ probabilities of $y$ being each of the $m$ different senders

$$ 
p(y|\theta) =
Cat(y|\theta) =
\prod_{k=1}^m \theta_k^{[y=k]}
$$

The distribution for the counts of words $x$ in a document is the multinomial distribution parameterized by $\phi_{yj}$, the probability of word $j$ appearing in documents from sender $y$.

$$ 
p(x|y,\phi) =
Mult(x|y,\phi) \propto 
\prod_{j=1}^d \phi_{yj}^{x_j} 
$$

For reasons that are not at all obvious now, but will come in handy later, we'll re-write this as

$$ p(x|y,\phi) \propto
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{x_j[k=y]} 
$$

For the priors on $\theta$ and $\phi$ we'll use dirichlet distributions as they're the conjugate priors for both the  categorical and the multinomial distribution. The priors are parameterized by the hyper parameters $\alpha$ and $\beta$.

$$ 
p(\theta) = 
Dir(\theta|\alpha) \propto
\prod_{k=1}^m \theta_k^{\alpha_k-1} 
$$

$$ 
p(\phi) =
\prod_{k=1}^m p(\phi_k) =
\prod_{k=1}^m Dir(\phi_k|\beta_k) \propto
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj}-1} 
$$

We'll find the posterior of the parameters first

$$
p(\theta, \phi | D) \propto p(\theta)p(\phi)p(D|\theta, \phi)
$$

Assuming the data is i.i.d

$$
p(\theta, \phi | D) \propto p(\theta)p(\phi)\prod_{i=1}^n p(x^{(i)}|y_i, \phi)p(y_i|\theta)
$$

Re-arranging

$$
p(\theta, \phi | D) \propto 
\left[
p(\theta)
\prod_{i=1}^n 
p(y_i|\theta)
\right]
\left[
p(\phi)
\prod_{i=1}^n 
p(x^{(i)}|y_i, \phi)
\right]
$$

We'll handle the leftmost bracket first. Inserting the definitions.


$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m \theta_k^{\alpha_k-1}
\prod_{i=1}^n 
\prod_{k=1}^m \theta_k^{[y=k]}
\right]
\left[
p(\phi)
\prod_{i=1}^n 
p(x^{(i)}|y_i, \phi)
\right]
$$



Introducing $c_k = \sum_{i=1}^n[y_i=k]$, the count of sender $k$ in the data set

$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m \theta_k^{\alpha_k-1}
\prod_{k=1}^m \theta_k^{c_k}
\right]
\left[
p(\phi)
\prod_{i=1}^n 
p(x^{(i)}|y_i, \phi)
\right]
$$

Joining the products

$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k -1}
\right]
\left[
p(\phi)
\prod_{i=1}^n 
p(x^{(i)}|y_i, \phi)
\right]
$$

And now the right hand bracket. Inserting the definitions

$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k -1}
\right]
\left[
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj}-1} 
\prod_{i=1}^n
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{x^{(i)}_j[k=y_i]} 
\right]
$$

Introducing $w_{kj} = \sum_{i=1}^n x^{(i)}_j[k=y_i]$, the sum of occurences of word $j$ in documents from sender $k$

$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k -1}
\right]
\left[
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj}-1} 
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{w_{kj}} 
\right]
$$

Joining the products

$$
p(\theta, \phi | D) \propto 
\left[
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k -1}
\right]
\left[
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj} + w_{kj} - 1} 
\right]
$$

We're interested in the posterior predictive distribution, i.e. given a new input $\tilde{x}$ what is the distribution of senders $\tilde{y}$, conditioned on the data set $D$.

$$ p(\tilde{y} | \tilde{x}, D) \propto \int \int p(\tilde{x}, \tilde{y} | \theta, \phi, D) p(\theta, \phi | D) d\theta d\phi $$

Re-arranging

$$
p(\tilde{y} | \tilde{x}, D) \propto 
\int 
p(\tilde{y} | \theta)
p(\theta | D) 
d\theta 
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

We'll handle the integral over $\theta$ first. Inserting the definitions

$$
p(\tilde{y} | \tilde{x}, D) \propto 
\int 
\prod_{k=1}^m \theta_k^{[\tilde{y}=k]}
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k -1}
d\theta 
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

Joining the products

$$
p(\tilde{y} | \tilde{x}, D) \propto 
\int 
\prod_{k=1}^m 
\theta_k^{\alpha_k + c_k + [\tilde{y}=k] - 1}
d\theta 
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

This is an integral over the un-normalized $Dir(\theta|\alpha_k + c_k + [\tilde{y}=k])$ distribution.

Using $\int \frac{1}{Z}p(x)dx = 1 \implies \int p(x) dx = Z$

$$
p(\tilde{y} | \tilde{x}, D) \propto 
\frac
{\prod_{k=1}^m\Gamma(\alpha_k + c_k + [\tilde{y}=k])}
{\Gamma \left( \sum_{k=1}^m \alpha_k + c_k + [\tilde{y}=k] \right)}
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

Using $\Gamma(x+1) = x\Gamma(x)$

$$
p(\tilde{y} | \tilde{x}, D) \propto 
\frac
{(\alpha_\tilde{y} + c_\tilde{y})}
{(\sum_{k=1}^m \alpha_k + c_k)}
\frac
{\prod_{k=1}^m\Gamma(\alpha_k + c_k)}
{\Gamma \left( \sum_{k=1}^m \alpha_k + c_k \right)}
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

Dropping the constant terms

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\int 
p(\tilde{x} | \tilde{y}, \phi)
p(\phi | D) 
d\phi
$$

Now for the integral over $\phi$. Inserting the definitions

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\int 
\left[
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\tilde{x}_j[k=\tilde{y}]} 
\right]
\left[
\prod_{k=1}^m
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj} + w_{kj} - 1} 
\right]
d\phi
$$

Joining the products, and moving the product over $k$ outside the integrals

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\prod_{k=1}^m
\int 
\left[
\prod_{j=1}^d 
\phi_{kj}^{\beta_{kj} + w_{kj} + \tilde{x}_j[k=\tilde{y}] - 1} 
\right]
d\phi
$$

This is an integral over the un-normalized $Dir(\theta_k|\beta_{k} + w_{k} + \tilde{x}_j[k=\tilde{y}])$ distribution

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\prod_{k=1}^m
\frac
{\prod_{j=1}^d\Gamma \left( \beta_{kj} + w_{kj} + \tilde{x}_j[k=\tilde{y}] \right)}
{\Gamma \left( \sum_{j=1}^d \beta_{kj} + w_{kj} + \tilde{x}_j[k=\tilde{y}] \right)}
$$

Splitting the sum in the denominator

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\prod_{k=1}^m
\frac
{\prod_{j=1}^d\Gamma \left( \beta_{kj} + w_{kj} + \tilde{x}_j[k=\tilde{y}] \right)}
{\Gamma \left( \sum_{j=1}^d (\beta_{kj} + w_{kj}) + \sum_{j=1}^d \tilde{x}_j[k=\tilde{y}] \right)}
$$

Using $\Gamma(x+n) = x^{(n)}\Gamma(x)$, where $x^{(n)} = x(x+1)...x(x+n-1)$ denotes the rising factorial

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\prod_{k=1}^m
\frac
{\prod_{j=1}^d (\beta_{\tilde{y}j} + w_{\tilde{y}j})^{(\tilde{x}_j[k=\tilde{y}])}}
{\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)^{(\sum_{j=1}^d \tilde{x}_j[k=\tilde{y}])}}
\frac
{\prod_{j=1}^d \Gamma \left( \beta_{kj} + w_{kj} \right)}
{\Gamma \left( \sum_{j=1}^d \beta_{kj} + w_{kj} \right)}
\frac
{\prod_{j=1}^d \Gamma \left( \beta_{kj} + w_{kj} \right)}
{\Gamma \left( \sum_{j=1}^d \beta_{kj} + w_{kj} \right)}
$$

Dropping the constant terms

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\prod_{k=1}^m
\frac
{\prod_{j=1}^d(\beta_{\tilde{y}j} + w_{\tilde{y}j})^{(\tilde{x}_j[k=\tilde{y}])}}
{\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)^{(\sum_{j=1}^d \tilde{x}_j[k=\tilde{y}])}}
$$

Using $x^{(0)} = 1$ by definition

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\frac
{\prod_{j=1}^d(\beta_{\tilde{y}j} + w_{\tilde{y}j})^{(\tilde{x}_j)}}
{\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)^{(\sum_{j=1}^d\tilde{x}_j)}}
$$

Using $x^{(n)} = \frac{\Gamma(x+n)}{\Gamma(x)}$


$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\frac
{
  \prod_{j=1}^d
  \frac{\Gamma((\beta_{\tilde{y}j} + w_{\tilde{y}j}) + \tilde{x}_j)}{\Gamma((\beta_{\tilde{y}j} + w_{\tilde{y}j}))}
}
{
  \frac{\Gamma(\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right) + \sum_{j=1}^d\tilde{x}_j)}{\Gamma(\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right))}
}
$$

Simplifying 

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\left[
\prod_{j=1}^d
\frac{\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j)}{\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j})}
\right]
\frac{\Gamma\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)}{\Gamma(\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right) + \sum_{j=1}^d\tilde{x}_j)}
$$

Simplifying 

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\left[
\prod_{j=1}^d
\frac{\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j)}{\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j})}
\right]
\frac{\Gamma\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)}
{\Gamma\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j\right)}
$$

Simplifying 

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\frac
{\prod_{j=1}^d\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j)}
{\Gamma\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j\right)}
\frac
{\Gamma\left(\sum_{j=1}^d \beta_{\tilde{y}j} + w_{\tilde{y}j}\right)}
{\prod_{j=1}^d\Gamma(\beta_{\tilde{y}j} + w_{\tilde{y}j})}
$$

Simplifying, where $B$ is the beta function

$$
p(\tilde{y} | \tilde{x}, D) \propto 
(\alpha_\tilde{y} + c_\tilde{y})
\frac
{B\left( \beta_{\tilde{y}j} + w_{\tilde{y}j} + \tilde{x}_j \right)}
{B\left(\beta_{\tilde{y}j} + w_{\tilde{y}j}\right)}
$$

In [3]:
%matplotlib inline
import numpy as np
import cPickle as pickle
import glob
import pystan
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rc('figure', figsize=(10, 16))

sv = CountVectorizer()
rv = CountVectorizer()
dv = CountVectorizer(max_df=1.0, min_df=0.01, max_features=10)

def load_data(pattern, train=False):
    docs = list()
    receivers = list()
    senders = list()
    for path in glob.glob("data/" + pattern):
        with open(path) as f:
            receivers.append(f.readline())
            senders.append(f.readline())
            docs.append(f.readline())

    if train:
        dv.fit(docs)
        sv.fit(senders)
        rv.fit(receivers)
        
    D = dv.transform(docs)
    S = sv.transform(senders)
    R = rv.transform(receivers)
    
    return D.toarray(), np.argmax(S.toarray(), axis=1), np.argmax(R.toarray(), axis=1)

In [9]:
train_X, train_s, train_r = load_data("00*", True)
val_X, val_s, val_r = load_data("ff*")
print train_X.shape
print train_s.shape

(842, 10)
(842,)


In [17]:
#scount = np.bincount(train_s).astype(np.float32)
#alpha = scount/scount.sum()
#beta = train_X.mean(axis=0)

alpha = np.ones(train_s.max()+1)
beta = np.ones(train_X.shape[1])/train_X.shape[1]

w = []
doc = []
for i in range(train_X.shape[0]):
    for j in range(train_X.shape[1]):
        for n in range(train_X[i,j]):
            doc.append(i+1)
            w.append(j+1)
            
w = np.array(w)
doc = np.array(doc)
K = alpha.size
V = beta.size
M = train_X.shape[0]
N = len(w)
s = train_s + 1

In [22]:
print K
print V
print M
print N

assert (w >= 1).all() and (w <= V).all()
assert (s >= 1).all() and (s <= K).all()
assert (doc >= 1).all() and (doc <= M).all()
            
data = {
    'K': K,
    'V': V,
    'M': M,
    'N': N,
    's': s,
    'w': w,
    'doc': doc,
    'alpha': alpha,
    'beta': beta
}

396
10
842
23267


In [23]:
code = """
// supervised naive Bayes

data {
  // training data
  int<lower=1> K;               // num senders
  int<lower=1> V;               // num words
  int<lower=1> M;               // num docs
  int<lower=1> N;               // number of words
  int<lower=1, upper=K> s[M];   // sender for doc m
  int<lower=1, upper=V> w[N];   // word n
  int<lower=1, upper=M> doc[N]; // doc ID for word n
  
  //hyper params
  //vector<lower=0>[K] alpha;   // sender prior
  //vector<lower=0>[V] beta;    // word prior
}
parameters {
  simplex[K] theta;             // sender prevalence
  simplex[V] phi[K];            // word dist for sender k
}
model {
  // priors
  //theta ~ dirichlet(alpha);
  //for (k in 1:K)  
  //  phi[k] ~ dirichlet(beta);
    
  // likelihood, including latent category
  for (m in 1:M)
    s[m] ~ categorical(theta);

  for (n in 1:N)
    w[n] ~ categorical(phi[s[doc[n]]]);

}
"""

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=1)
params = fit.extract()
print fit

Inference for Stan model: anon_model_a678076fb9992286e7b405d0e30893a2.
1 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=500.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
theta[0]   3.2e-3  1.3e-4 1.7e-3 7.8e-4 2.0e-3 2.9e-3 4.2e-3 7.2e-3  167.0    1.0
theta[1]   1.6e-3  8.3e-5 1.1e-3 2.4e-4 7.7e-4 1.4e-3 2.2e-3 4.2e-3  167.0    1.0
theta[2]   1.6e-3  9.0e-5 1.2e-3 1.9e-4 7.2e-4 1.3e-3 2.2e-3 4.4e-3  167.0    1.0
theta[3]   3.2e-3  1.2e-4 1.6e-3 9.4e-4 2.1e-3 2.9e-3 4.2e-3 6.8e-3  167.0    1.0
theta[4]   1.6e-3  8.5e-5 1.1e-3 2.2e-4 7.9e-4 1.4e-3 2.3e-3 4.2e-3  167.0    1.0
theta[5]   3.2e-3  1.1e-4 1.4e-3 1.1e-3 2.2e-3 3.0e-3 4.0e-3 6.4e-3  167.0    1.0
theta[6]   3.3e-3  1.3e-4 1.7e-3 8.0e-4 2.0e-3 3.0e-3 4.2e-3 7.0e-3  167.0    1.0
theta[7]   1.7e-3  8.6e-5 1.1e-3 2.7e-4 8.9e-4 1.4e-3 2.1e-3 4.7e-3  167.0    1.0
theta[8]   3.3e-3  1.3e-4 1.6e-3 9.9e-4 2.2e-3 3.0e-3 4.2e-3 7.3e-3  167.0  

In [ ]:
eq = (val_s==pred_s)
neq = (val_s!=pred_s)
nse = val_s==0
print prob.shape
print prob[eq].shape
print prob[neq].shape
print prob[nse].shape
plt.subplot(3,1,1)
_=plt.hist(prob[eq].max(axis=1), bins=100, normed=True)
print (prob[eq].max(axis=1)>0.99).mean()
plt.subplot(3,1,2)
_=plt.hist(prob[neq].max(axis=1), bins=100, normed=True)
print (prob[neq].max(axis=1)<0.99).mean()
plt.subplot(3,1,3)
_=plt.hist(prob[nse].max(axis=1), bins=100, normed=True)
print (prob[nse].max(axis=1)<0.99).mean()